# Design Agent 통합 노트북

이 노트북은 기존의 `design.ipynb`와 `designer.ipynb`를 합쳐 Design Agent 워크플로우를 구성합니다.

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

In [ ]:
from langchain_core.tools import tool
from typing import List, Dict

# 도구 생성
@tool
def color_gen(query: str) -> List[Dict[str, str]]:
    """generate color based on the query"""

    if query == "warm":
        return [{"red": "#FF0000"}, {"orange": "#FFA500"}, {"yellow": "#FFFF00"}]

    return [{"blue": "#0000FF"}, {"green": "#008000"}, {"purple": "#800080"}]

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
tools = [color_gen]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)


########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"]) ]}

# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools=tools)
# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########
# tools > chatbot
graph_builder.add_edge("tools", "chatbot")
# START > chatbot
graph_builder.add_edge(START, "chatbot")
# chatbot > END
graph_builder.add_edge("chatbot", END)

In [ ]:
# 그래프 빌더 컴파일
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from langchain_teddynote.graphs import visualize_graph

# 그래프 시각화
visualize_graph(graph)

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

In [ ]:
# 첫 질문
question = (
    "안녕하세요 당신은 누구인가요?"
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

In [ ]:
# 이어지는 질문
question = "따뜻한 컵을 디자인하기 위해 어떤 색상을 사용해야 할까요?"

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

---

# Designer Agent 예제

디자이너 에이전트가 쿼리에 맞게 디자인 계획을 세우고 계획을 달성하는 과정에서 ColorGen이라는 도구를 사용합니다.

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

In [ ]:
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
from langchain_core.tools import tool
from typing import List, Dict

# 도구 생성
@tool
def color_gen(query: str) -> List[Dict[str, str]]:
    """generate color based on the query"""

    if query == "warm":
        return [{"red": "#FF0000"}, {"orange": "#FFA500"}, {"yellow": "#FFFF00"}]

    return [{"blue": "#0000FF"}, {"green": "#008000"}, {"purple": "#800080"}]

@tool
def font_gen(query: str) -> List[Dict[str, str]]:
    """generate font based on the query"""

    if query == "warm":
        return [{"Arial": "sans-serif"}, {"Times New Roman": "serif"}, {"Courier New": "monospace"}]

    return [{"Helvetica": "sans-serif"}, {"Georgia": "serif"}, {"Comic Sans MS": "cursive"}]

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

# 도구 리스트 생성
tools = [color_gen, font_gen]

# ToolNode 초기화
tool_node = ToolNode(tools)

In [ ]:
from langchain_core.messages import AIMessage

# 단일 도구 호출을 포함하는 AI 메시지 객체 생성
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "color_gen",
            "args": {"query": "warm"},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

# 도구 노드를 통한 메시지 처리
tool_node.invoke({"messages": [message_with_single_tool_call]})

In [ ]:
from langchain_openai import ChatOpenAI

# LLM 모델 초기화 및 도구 바인딩
model_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

In [ ]:
# 모델 도구 호출 테스트
model_with_tools.invoke("따뜻한 그림에는 어떤 색이 어울릴까?").tool_calls

In [ ]:
# 도구 노드를 통한 메시지 처리 및 응답 생성
tool_node.invoke({"messages": [model_with_tools.invoke("따뜻한 그림에는 어떤 색이 어울릴까?") ]})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage

# 디자이너 콜 함수 정의
def call_designer(messages: List[BaseMessage]) -> dict:
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a professional designer whose expertise lies in creating a wide range of promotional materials..."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ])
    chain = prompt | model_with_tools
    return chain.invoke({"messages": messages})

# 콜 예제
call_designer([("user", "안녕하세요? 따뜻한 톤의 그림을 위해서는 어떠한 색이 어울릴까요?")])

In [ ]:
from langgraph.graph import StateGraph, START, END

# 디자이너 노드 정의
def designer_node(state: State) -> State:
    ai_response = call_designer(state["messages"])
    return {"messages": [AIMessage(ai_response)]}

# 워크플로우 그래프 초기화
workflow = StateGraph(State)
workflow.add_node("designer", designer_node)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "designer")
workflow.add_conditional_edges("designer", tools_condition)
workflow.add_edge("tools", "designer")
workflow.add_edge("designer", END)
app = workflow.compile()

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
inputs = {"messages": [HumanMessage(content="안녕하세요? 따뜻한 톤의 그림을 위해서는 어떠한 색이 어울릴까요?") ]}
stream_graph(app, inputs, config)